In [16]:
# need to read dataset from huggingface

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
import os
import time
import random
import math
import copy
import json
import re
from sklearn.preprocessing import LabelEncoder
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel
from sklearn.cluster import KMeans
from torch.utils.data import DataLoader


# huggingface datasets
# ! pip install transformers

In [2]:
def data_loader(index):
    url = 'shahidul034/text_summarization_dataset'
    url = url + str(index)
    dataset = load_dataset(url)

    path = 'text_summarization_dataset' + str(index) + '.csv'
    dataset['train'].to_csv(path)
    df = pd.read_csv(path)
    



    

In [ ]:
# initialize dataframe
df = pd.DataFrame(columns=['title', 'content'])
for i in range(1, 10):
    data_loader(i)
    df
# index = 2

# dataset = load_dataset(url)



In [ ]:
df = pd.read_csv('text_summarization_dataset8.csv')
df

In [5]:
url = 'shahidul034/text_summarization_dataset'
url = url + "2"
dataset = load_dataset(url)

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'content'],
        num_rows: 105252
    })
})

In [6]:
dataset['train'][2]

{'title': 'শাহজালালে বিমানের টয়লেটে মিলল ৪০ স্বর্ণের বার',
 'content': ' কাস্টমস কর্তৃপক্ষ সূত্র জানায়, কাস্টমস গোয়েন্দা ও তদন্ত অধিদপ্তরের মহাপরিচালকের কাছে সংবাদ আসে সংযুক্ত আরব আমিরাতের আবুধাবি থেকে চট্টগ্রাম হয়ে আসা বিমান বাংলাদেশ এয়ারলাইনসের বিমানে অবৈধভাবে চোরাচালানকৃত স্বর্ণ বহন করা হচ্ছে। এর প্রেক্ষিতে কাস্টমস গোয়েন্দা ও তদন্ত অধিদপ্তরের যুগ্ম মহাপরিচালক আরেফিন খানের নির্দেশে উপপরিচালক আহমেদুর রেজা চৌধুরীর নেতৃত্বে কাস্টমস গোয়েন্দার একটি দল ১০ নম্বর বোর্ডিং ব্রিজের সামনে অবস্থান নেয়।পরে কাস্টমস গোয়েন্দা দল বিমানের ভেতরে প্রবেশ করে এবং বিমানের টয়লেট থেকে বিশেষভাবে লুকিয়ে রাখা ৪০টি স্বর্ণবার উদ্ধার করে। যার মোট ওজন চর কেজি ৬৪০ গ্রাম। আটক স্বর্ণের বাজারমূল্য আনুমানিক তিন কোটি ২৪ লাখ ৮০ হাজার টাকা। এ বিষয়ে বিমানবন্দর থানায় মামলা দায়েরের প্রক্রিয়া চলছে।'}

In [11]:
bnbert_tokenizer = AutoTokenizer.from_pretrained("sagorsarker/bangla-bert-base")
# text = "আমি বাংলায় গান গাই।"
# bnbert_tokenizer.tokenize(text)
# ['আমি', 'বাংলা', '##য', 'গান', 'গাই', '।']

text = dataset['train'][1]['content']
text
# 'শাহজালাল আন্তর্জাতিক বিমানবন্দরে বিমান বাংলাদেশ এয়ারলাইনসের একটি ফ্লাইটের টয়লেট থেকে বিশেষভাবে লুকিয়ে রাখা ৪০টি স্বর্ণের বার উদ্ধার করা হয়েছে। আজ সোমবার দুপুর সাড়ে ১২টার দিকে এই স্বর্ণ উদ্ধার করেন কাস্টমস গোয়েন্দা ও তদন্ত অধিদপ্তরের কর্মকর্তারা।প্রতিটি বারের ওজন ১১৬ গ্রাম। মোট ওজন চার কেজি ৬৪০ গ্রাম বলে জানা গেছে'
bnbert_tokenizer.tokenize(text)

config.json:   0%|          | 0.00/491 [00:00<?, ?B/s]

c:\Users\Sayeed\anaconda3\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Sayeed\.cache\huggingface\hub\models--sagorsarker--bangla-bert-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/2.24M [00:00<?, ?B/s]

['শাহজালাল',
 'আনত',
 '##র',
 '##জাতিক',
 'বিমান',
 '##বন',
 '##দরে',
 'বিমান',
 'বাংলাদেশ',
 'এ',
 '##যার',
 '##লাইন',
 '##সের',
 'একটি',
 'ফলা',
 '##ইট',
 '##ের',
 'ট',
 '##য',
 '##লেট',
 'থেকে',
 'বিশেষভাবে',
 'লকি',
 '##যে',
 'রাখা',
 '[UNK]',
 'সব',
 '##র',
 '##ণের',
 'বার',
 'উদ',
 '##ধার',
 'করা',
 'হযেছে',
 '।',
 'আজ',
 'সে',
 '##াম',
 '##বার',
 'দপ',
 '##র',
 'সা',
 '##ডে',
 '[UNK]',
 'দিকে',
 'এই',
 'সব',
 '##র',
 '##ণ',
 'উদ',
 '##ধার',
 'করেন',
 'কাস',
 '##টম',
 '##স',
 'গে',
 '##ায',
 '##েন',
 '##দা',
 'ও',
 'তদ',
 '##ন',
 '##ত',
 'অধি',
 '##দ',
 '##পত',
 '##রে',
 '##র',
 'করম',
 '##কর',
 '##তারা',
 '।',
 'পরত',
 '##িটি',
 'বারের',
 'ওজন',
 '[UNK]',
 'গর',
 '##াম',
 '।',
 'মে',
 '##াট',
 'ওজন',
 'চার',
 'কেজি',
 '[UNK]',
 'গর',
 '##াম',
 'বলে',
 'জানা',
 'গেছে']

In [12]:
texts = dataset['train']['content']
# texts
labels = dataset['train']['title']
# labels

In [13]:
# Assuming you have your training data in two lists: texts (the texts) and labels (the corresponding labels)
inputs = bnbert_tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)

In [15]:


# Assuming labels is your list of string labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

labels_tensor = torch.tensor(labels_encoded)

In [17]:
tokenizer = AutoTokenizer.from_pretrained("sagorsarker/bangla-bert-base")
model = AutoModel.from_pretrained("sagorsarker/bangla-bert-base")


inputs

model.safetensors:   0%|          | 0.00/660M [00:00<?, ?B/s]

{'input_ids': tensor([[  101, 99459, 13674,  ...,     0,     0,     0],
        [  101,  8293, 40230,  ...,     0,     0,     0],
        [  101, 24297, 42032,  ...,     0,     0,     0],
        ...,
        [  101,  2045, 13395,  ...,     0,     0,     0],
        [  101, 74056, 12499,  ...,     0,     0,     0],
        [  101,  2224, 15749,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [ ]:

def collate_fn(batch):
    keys = batch[0].keys()
    return {key: torch.stack([d[key] for d in batch]) for key in keys}

embeddings = []
for i in range(0, len(texts), 8):
    batch = texts[i:i+8]
    inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    embeddings.append(outputs.last_hidden_state[:, 0, :].detach().numpy())

In [26]:
embeddings

[array([[ 6.3048534e-02, -1.0059899e+00, -1.0187241e-02, ...,
          4.0842444e-01, -1.0902221e+00, -1.1599921e-03],
        [ 1.1306834e-01,  4.4198233e-01,  4.5221740e-01, ...,
          6.6738111e-01, -3.2191163e-01, -2.4554427e-01],
        [ 1.7841166e-01, -1.4506301e+00,  7.2876287e-01, ...,
         -4.3964195e-01, -6.6702574e-01, -8.2468855e-01],
        ...,
        [ 1.1271700e+00, -1.1603340e+00, -2.4218202e-01, ...,
         -4.8258412e-01, -3.9993894e-01, -8.3991635e-01],
        [ 1.3680558e+00, -1.5031849e+00, -1.2665623e-01, ...,
         -2.4286543e-01, -2.5102702e-01, -1.7384568e+00],
        [ 1.1299088e+00, -3.2317016e-01, -1.2771574e-01, ...,
         -1.4223233e+00,  1.9148304e+00, -2.4589908e+00]], dtype=float32),
 array([[ 1.3854376 , -0.96453416, -0.3042114 , ..., -1.7103598 ,
          0.91821265, -2.5589397 ],
        [ 0.64063954,  1.4917219 , -0.48364592, ...,  0.74736273,
         -0.89910674, -0.4111039 ],
        [ 0.32774276,  0.6031021 ,  1.4491438 

In [18]:
# Get sentence embeddings
with torch.no_grad():
    embeddings = model(**inputs).last_hidden_state[:, 0, :]

# Perform clustering
kmeans = KMeans(n_clusters=3)  # Change the number of clusters as needed
clusters = kmeans.fit_predict(embeddings)

# Now, 'clusters' contains the cluster assignments for each sentence

RuntimeError: [enforce fail at alloc_cpu.cpp:80] data. DefaultCPUAllocator: not enough memory: you tried to allocate 165547081728 bytes.